In [13]:
import bigframes.pandas as bpd
import pandas as pd

In [14]:
bpd.options.bigquery.project = "ajp-ds-gcp"
bpd.options.bigquery.location = "US"

In [ ]:
table: str = "dsongcp.flights_raw"

df_flights_raw: bpd.DataFrame = bpd.read_gbq(query=table)
df_flights_raw2: bpd.DataFrame = df_flights_raw.copy()
pd.testing.assert_frame_equal(df_flights_raw, df_flights_raw2)

In [9]:
def transforms(df_raw: bpd.DataFrame) -> bpd.DataFrame:
    columns_selected: dict = {
        "FlightDate": "flight_date",
        "Reporting_Airline": "unique_carrier",
        "OriginAirportSeqID": "origin_airport_seq_id",
        "Origin": "origin",
        "DestAirportSeqID": "dest_airport_seq_id",
        "Dest": "dest",
        "CRSDepTime": "crs_dep_time",
        "DepTime": "dep_time",
        "DepDelay": "dep_delay",
        "TaxiOut": "taxi_out",
        "WheelsOff": "wheels_off",
        "WheelsOn": "wheels_on",
        "TaxiIn": "taxi_in",
        "CRSArrTime": "crs_arr_time",
        "ArrTime": "arr_time",
        "ArrDelay": "arr_delay",
        "Cancelled": "cancelled",
        "Diverted": "diverted",
        "Distance": "distance",
    }

    # Doing explicit casts with a list of columns
    # The conventional dictionary method doesn't work with Bigframes
    columns_float: list = [
        "dep_delay",
        "taxi_out",
        "taxi_in",
        "arr_delay",
    ]

    map_bool = {
        "1.00": True,
        "0.00": False
    }    
    
    columns_bool = ["cancelled", "diverted"]
    df_transformed: bpd.DataFrame = df_raw.copy()
    df_transformed = df_transformed[columns_selected.keys()].rename(columns=columns_selected)

    for col in columns_bool:
        df_transformed[col] = df_transformed[col].map(map_bool)
    
    for col in columns_float:
        df_transformed[col] = df_transformed[col].astype("Float64")

    return df_transformed

    
df_flights_transformed: bpd.DataFrame = transforms(df_raw=df_flights_raw)

In [11]:
df_flights_transformed.to_gbq("dsongcp.flights", if_exists="replace")

HTML(value='Query job b22123d1-c3b9-46c3-acf3-4210db77ac53 is RUNNING. <a target="_blank" href="https://consol…